## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [1]:
# 1

import pandas as pd
import numpy as np

file_name = ("/Users/piman/LaboratoryOfComputationalPhysics_Y3/LaboratoryOfComputationalPhysics_Y3/data_000637.txt")
N=10000
D = pd.read_csv(file_name,nrows=N)
D
     

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
9995,1,0,61,3869200267,696,11
9996,1,0,60,3869200267,701,5
9997,1,0,59,3869200267,707,23
9998,1,0,63,3869200267,706,15


In [2]:
# 2


FirstValue = D.iloc[0]['ORBIT_CNT']
LastValue = D.iloc[-1]['ORBIT_CNT']

res = []
for v in range(FirstValue ,LastValue+1):
    res.append(D[D['ORBIT_CNT'] == v]['BX_COUNTER'].sum())

print("on average number of {0} BX (x) were pass in each ORBIT" .format(np.mean(np.array(res))))    






on average number of 179365.0792079208 BX (x) were pass in each ORBIT


In [3]:
# 3

AllData = pd.read_csv(file_name)
print("data take duration :: {0} ns" .format(25.0 * AllData["BX_COUNTER"].sum()))

data take duration :: 58362964475.0 ns


In [4]:
# 4

D["AbsTime"] = D["BX_COUNTER"] * 25.0
D

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,AbsTime
0,1,0,123,3869200167,2374,26,59350.0
1,1,0,124,3869200167,2374,27,59350.0
2,1,0,63,3869200167,2553,28,63825.0
3,1,0,64,3869200167,2558,19,63950.0
4,1,0,64,3869200167,2760,25,69000.0
...,...,...,...,...,...,...,...
9995,1,0,61,3869200267,696,11,17400.0
9996,1,0,60,3869200267,701,5,17525.0
9997,1,0,59,3869200267,707,23,17675.0
9998,1,0,63,3869200267,706,15,17650.0


In [5]:
# 5

D["HEAD"] = np.random.randint(low=0,high=2,size=D.shape[0])
D


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,AbsTime
0,0,0,123,3869200167,2374,26,59350.0
1,0,0,124,3869200167,2374,27,59350.0
2,1,0,63,3869200167,2553,28,63825.0
3,1,0,64,3869200167,2558,19,63950.0
4,1,0,64,3869200167,2760,25,69000.0
...,...,...,...,...,...,...,...
9995,0,0,61,3869200267,696,11,17400.0
9996,1,0,60,3869200267,701,5,17525.0
9997,1,0,59,3869200267,707,23,17675.0
9998,1,0,63,3869200267,706,15,17650.0


In [6]:
# 6

DwoZ = D[D["HEAD"] == 1].copy()

In [9]:
#  8

GroupedD = D.groupby("TDC_CHANNEL").sum()
GroupedD.sort_values(by="BX_COUNTER").iloc[-3:]



,HEAD,FPGA,ORBIT_CNT,BX_COUNTER,TDC_MEAS,AbsTime
TDC_CHANNEL,,,,,,
63,279,3,1930730908900,930809,7652,23270225.0
64,253,11,1899777307239,934112,7115,23352800.0
139,448,261,3327512186268,1538908,0,38472700.0


In [17]:
# 9

D.groupby("ORBIT_CNT").sum().shape[0]
D[D["TDC_CHANNEL"] == 139].groupby("ORBIT_CNT").sum().shape[0]

101

In [16]:
print(D.groupby("ORBIT_CNT").sum())

            HEAD  FPGA  TDC_CHANNEL  BX_COUNTER  TDC_MEAS    AbsTime
ORBIT_CNT                                                           
3869200167    17     6         2683      124133       619  3103325.0
3869200168    43    36         4376       97201      1134  2430025.0
3869200169    72    37         6727      144343      1688  3608575.0
3869200170    43    32         5183      217462      1308  5436550.0
3869200171    59    30         6562      228822      1482  5720550.0
...          ...   ...          ...         ...       ...        ...
3869200263    59    33         6557      279850      1686  6996250.0
3869200264    37    19         5311      158363       952  3959075.0
3869200265    49    25         4549      148300      1211  3707500.0
3869200266    63    33         7892      166816      1970  4170400.0
3869200267    10     0         1094       11021       249   275525.0

[101 rows x 6 columns]


In [18]:

print(D[D["TDC_CHANNEL"] == 139].groupby("ORBIT_CNT").sum())

            HEAD  FPGA  TDC_CHANNEL  BX_COUNTER  TDC_MEAS   AbsTime
ORBIT_CNT                                                          
3869200167     1     1          556       11844         0  296100.0
3869200168     2     3         1112       10211         0  255275.0
3869200169     7     2         1529       12820         0  320500.0
3869200170     4     3         1251       19598         0  489950.0
3869200171     3     2         1251       18432         0  460800.0
...          ...   ...          ...         ...       ...       ...
3869200263     5     3          834       13980         0  349500.0
3869200264     5     2          973       16846         0  421150.0
3869200265     3     2          973       11645         0  291125.0
3869200266     7     4         1390       11435         0  285875.0
3869200267     0     0          139         616         0   15400.0

[101 rows x 6 columns]
